In [34]:
import pandas as pd
import numpy as np

# ----------------------------
# 1️⃣ CHARGEMENT DES DONNÉES
# ----------------------------
file_communes = "donnees_communes_filtrees_2023.xlsx"
file_departements = "donnees_departement_2023.xlsx"
file_regions = "donnees_region_2023.xlsx"

df_communes = pd.read_excel(file_communes)
df_departements = pd.read_excel(file_departements)
df_regions = pd.read_excel(file_regions)

# ----------------------------
# 2️⃣ SÉLECTION DES CRITÈRES
# ----------------------------
criteria_communes = ["fprod", "fcaf", "fcafn", "febf", "fdette", "fequip"]
criteria_departements = ["ftpf", "fcaf", "fcnr", "febf", "fdba", "fded"]
criteria_regions = ["ftpf", "fcaf", "fcnr", "febf", "fdba", "fded"]

# Columns identifying the territory
territory_columns = {
    "communes": "inom",
    "departements": "lbudg",
    "regions": "lbudg"
}

def filter_and_clean(df, criteria, territory_column):
    """Filtrer les données et nettoyer les valeurs NaN."""
    # Keep only relevant columns
    df = df[[territory_column] + criteria].dropna()
    return df

df_communes = filter_and_clean(df_communes, criteria_communes, territory_columns["communes"])
df_departements = filter_and_clean(df_departements, criteria_departements, territory_columns["departements"])
df_regions = filter_and_clean(df_regions, criteria_regions, territory_columns["regions"])

# ----------------------------
# 3️⃣ NORMALISATION DES DONNÉES
# ----------------------------
def normalize(df, excluded_column):
    """Normalise les données entre 0 et 1 (Min-Max), en excluant une colonne."""
    numeric_df = df.drop(columns=[excluded_column])
    normalized_df = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())
    normalized_df[excluded_column] = df[excluded_column]
    return normalized_df

df_communes = normalize(df_communes, territory_columns["communes"])
df_departements = normalize(df_departements, territory_columns["departements"])
df_regions = normalize(df_regions, territory_columns["regions"])

# ----------------------------
# 4️⃣ FONCTION ELECTRE AVEC POIDS ET SEUILS
# ----------------------------
def electre(df, weights, indifference_threshold, preference_threshold, veto_threshold):
    """Applique la méthode ELECTRE avec poids et seuils personnalisables."""
    n = df.shape[0]
    concordance = np.zeros((n, n))
    discordance = np.zeros((n, n))

    numeric_df = df.drop(columns=[df.columns[-1]])

    for i in range(n):
        for j in range(n):
            if i != j:
                concordance[i, j] = np.sum(weights * (numeric_df.iloc[i] >= numeric_df.iloc[j])) / np.sum(weights)
                discordance[i, j] = np.max(np.abs(numeric_df.iloc[i] - numeric_df.iloc[j]))

                # Application des seuils
                if discordance[i, j] < indifference_threshold:
                    discordance[i, j] = 0
                elif discordance[i, j] > veto_threshold:
                    discordance[i, j] = 1

    return concordance, discordance

# ----------------------------
# 5️⃣ CLASSEMENT FINAL DES TERRITOIRES
# ----------------------------
def rank_territories(concordance, discordance):
    """Classement final basé sur ELECTRE."""
    scores = np.sum(concordance - discordance, axis=1)
    return np.argsort(scores)[::-1]  # Classement décroissant

def evaluate_sfil_preferences(territories_df, selected_territories, weights, indifference_threshold, preference_threshold, veto_threshold, territory_column):
    """Applique ELECTRE sur une liste personnalisée de territoires avec préférences de SFIL."""
    df_filtered = territories_df[territories_df[territory_column].isin(selected_territories)]
    if df_filtered.empty:
        return pd.DataFrame([], columns=territories_df.columns)
    concordance, discordance = electre(df_filtered, weights, indifference_threshold, preference_threshold, veto_threshold)
    ranking = rank_territories(concordance, discordance)
    return df_filtered.iloc[ranking]

# ----------------------------
# 6️⃣ TEST AVEC DES POIDS PAR DÉFAULT POUR SFIL
# ----------------------------
default_weights = np.array([0.2, 0.2, 0.15, 0.15, 0.2, 0.1])
default_indifference_threshold = 0.05
default_preference_threshold = 0.15
default_veto_threshold = 0.4

# Exemple d'utilisation avec une sélection spécifique de territoires
selected_communes = ["TROYES", "PAU"]
selected_departements = ["DEP AISNE", "DEP DES HAUTES-ALPES", "DEP MARNE"]
selected_regions = ["REG BRETAGNE", "REG ILE-DE-FRANCE"]





In [35]:
print("\n🔹 CLASSEMENT DES COMMUNES")
(evaluate_sfil_preferences(df_communes, selected_communes, default_weights, default_indifference_threshold, default_preference_threshold, default_veto_threshold, territory_columns["communes"]))


🔹 CLASSEMENT DES COMMUNES


,fprod,fcaf,fcafn,febf,fdette,fequip,inom
8,0.242935,0.386976,0.492540,0.365139,0.317903,0.377308,PAU
7,0.224782,0.306609,0.447684,0.284015,0.195738,0.484021,TROYES


In [36]:

print("\n🔹 CLASSEMENT DES DÉPARTEMENTS")
(evaluate_sfil_preferences(df_departements, selected_departements, default_weights, default_indifference_threshold, default_preference_threshold, default_veto_threshold, territory_columns["departements"]))


🔹 CLASSEMENT DES DÉPARTEMENTS


,ftpf,fcaf,fcnr,febf,fdba,fded,lbudg
1,0.259722,0.838569,0.823714,0.799219,0.342686,0.552004,DEP DES HAUTES-ALPES
0,0.105866,0.071040,0.047529,0.101987,0.321256,0.017461,DEP AISNE
18,0.027093,0.107420,0.146650,0.089710,0.091030,0.053482,DEP MARNE


In [37]:

print("\n🔹 CLASSEMENT DES RÉGIONS")
(evaluate_sfil_preferences(df_regions, selected_regions, default_weights, default_indifference_threshold, default_preference_threshold, default_veto_threshold, territory_columns["regions"]))


🔹 CLASSEMENT DES RÉGIONS


,ftpf,fcaf,fcnr,febf,fdba,fded,lbudg
7,0.110341,0.228126,0.337377,0.179164,0.137778,0.036242,REG BRETAGNE
0,0.000000,0.180061,0.173369,0.133323,0.224354,0.041286,REG ILE-DE-FRANCE


In [ ]:
df_communes

,fprod,fcaf,fcafn,febf,fdette,fequip,inom
0,0.323106,0.367004,0.462474,0.366669,0.397437,0.565308,ROSNY-SOUS-BOIS
1,0.205085,0.397145,0.556589,0.349800,0.125674,0.296617,MARIGNANE
2,0.123613,0.282870,0.479069,0.239195,0.089244,0.356382,MEYZIEU
3,0.260151,0.159844,0.299692,0.183717,0.314442,0.216746,BEZONS
4,0.244643,0.322466,0.464447,0.352517,0.458477,0.291277,PLAISIR
...,...,...,...,...,...,...,...
282,0.148504,0.269882,0.441716,0.254544,0.149884,0.220417,DIJON
283,0.204886,0.402914,0.463208,0.420164,0.278961,0.321277,REIMS
284,0.235509,0.240589,0.246032,0.291151,0.591786,0.262170,FREJUS
285,0.355794,0.213214,0.312866,0.267196,0.370264,0.089855,VITRY-SUR-SEINE
